# 4. Create Azure Virtual Machine as IoT Edge device
Recall that in [03_create_azure_services](03_create_azure_services.ipynb) we previously created an IoT Edge resource in Azure IoT Hub. Because we do not have a physical IoT Edge device to run LVA, we will be creating and configuring a virtual machine (VM) in this section to act as our IoT Edge device.

## 4.1. Choose a VM Size
When creating the VM, you have the option of choosing a GPU-accelerated VM or a CPU-accelerated VM. Choose the option that works best for your future needs and requriments; rest assured, we have samples for both types. 

For this sample, we suggest choosing either a Standard_NC6 (GPU tier) VM or a Standard_DS3_v2 (CPU tier) VM. For more information on other types of VMs or different sized machines available in Azure, visit the documentation about [virtual machine types and sizes](https://docs.microsoft.com/en-us/azure/virtual-machines/sizes-general).

In [1]:
# To choose which type of VM you will be creating, comment out the variable which will not be in use.
vm_size="Standard_NC6" #GPU tier VM
vm_size="Standard_DS3_v2" #CPU tier VM

## 4.2. Get Global Variables
First, we will read the global variables which were previously stored in the .env file.

In [ ]:
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

resourceLocation = get_key(envPath, "resourceLocation")
resourceGroupName = get_key(envPath, "resourceGroupName")
iotEdgeDeviceConnString = get_key(envPath, "iotEdgeDeviceConnString")
# Use the IoT Edge Device ID as the name of the VM
iotDeviceId = get_key(envPath, "iotDeviceId")

# Get the current username, which will be used to create SSH keys for connecting to the VM
import os
userName = os.environ['USER']

## 4.3. Create a Virtual Machine
The code snippets below will execute a shell script to create an Ubuntu VM in your Azure subscription. 

Before starting, we will be setting the parameters to be used for creating the VM. For more details on the process of creating a VM using Azure, see the tutorial on creating a [Linux VM on Azure CLI](https://docs.microsoft.com/en-us/azure/virtual-machines/linux/quick-create-cli) or the tutorial on creating a [Windows VM on Azure CLI](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/quick-create-cli).

In [ ]:
dns_name = iotDeviceId
public_ip_name = iotDeviceId + 'publicip'
vnet_name=iotDeviceId + 'vnet'
subnet_name=iotDeviceId + 'subnet'
vnet_prefix="192.168.0.0/16"
subnet_name="FrontEnd"
subnet_prefix="192.168.1.0/24"
nsg_name=iotDeviceId + 'nsg'
nic_name=iotDeviceId + 'nic'

# Static DNS name of the VM
vm_dns_name= iotDeviceId + "." + resourceLocation + ".cloudapp.azure.com"

In [ ]:
%%bash --out output -s "$iotDeviceId" "$userName" "$vm_dns_name"

# -----------------------
# Create SSH Keys - !!! DONT RUN THIS CELL IN CASE YOU ALREADY CREATED THE VM. IT WILL RESET THE PUBLIC KEYS!!!
# -----------------------
# Create public/private ssh keys.
yes y | ssh-keygen -b 2048 -t rsa -f ~/.ssh/$1"_id_rsa" -C $2@$3 -q -N '' >/dev/null

# Remove prev. trusted host with same name (IP may be changed)
if [ -e ~/.ssh/known_hosts ]; then
ssh-keygen -f ~/.ssh/known_hosts -R $3
fi

In [ ]:
%%bash -s "$resourceGroupName" "$resourceLocation" "$public_ip_name" "$dns_name" "$vnet_name" "$vnet_prefix" "$subnet_name" "$subnet_prefix" "$nsg_name" "$nic_name" "$iotDeviceId" "$vm_size"

# Create resource group
az group create                                                                             \
    --name "$1"                                                                             \
    --location "$2"

# Create a public IP address resource with a static IP address
az network public-ip create                                                                 \
   --name "$3"                                                                              \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --allocation-method Static                                                               \
   --dns-name "$4"                                                                                   

# Create a virtual network with one subnet
az network vnet create                                                                      \
   --name "$5"                                                                              \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --address-prefix "$6"                                                                    \
   --subnet-name "$7"                                                                       \
   --subnet-prefix "$8"                                                                         


az network nsg create                                                                       \
   --name "$9"                                                                              \
   --resource-group "$1"                                                                  

# Open SSH port
az network nsg rule create                                                                  \
   --resource-group "$1"                                                                    \
   --nsg-name "$9"                                                                          \
   --name "Default SSH"                                                                     \
   --destination-port-ranges 22                                                             \
   --protocol Tcp                                                                           \
   --access Allow                                                                           \
   --priority 1020

# Create a network interface connected to the VNet with a static private IP address 
# and associate the public IP address resource to the NIC.
az network nic create                                                                       \
   --name "${10}"                                                                           \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --subnet "$7"                                                                            \
   --vnet-name "$5"                                                                         \
   --public-ip-address "$3"                                                                 \
   --network-security-group "$9"                                                                     

# Create the VM
# For image param: Ubuntu 16.04-LTS image -> https://docs.microsoft.com/en-us/azure/virtual-machines/linux/cli-ps-findimage
# Instead of Password login, we are uploading Private SSH key. Update the path accrodingly if needed
az vm create                                                                                \
   --name "${11}"                                                                           \
   --resource-group "$1"                                                                    \
   --location "$2"                                                                          \
   --storage-sku Standard_LRS                                                               \
   --os-disk-name "${11}""_osdisk"                                                          \
   --image "Canonical:UbuntuServer:18.04-LTS:latest"                                        \
   --size "${12}"                                                                           \
   --nics "${10}"                                                                           \
   --authentication-type ssh                                                                \
   --ssh-key-value "$(< ~/.ssh/"${11}"_id_rsa.pub)"

In [ ]:
%%bash -s "$iotDeviceId" "$userName" "$vm_dns_name"

# Install ssh keys to remote machine
ssh -i ~/.ssh/$1"_id_rsa" -o "StrictHostKeyChecking no" $2@$3 "echo 'SSH key transferred.'"

## 4.4. Restart the VM

In [ ]:
%%bash  -s "$iotDeviceId" "$userName" "$vm_dns_name"
ssh -i ~/.ssh/$1"_id_rsa" $2@$3 << EOF
sudo -b bash -c 'sleep 5; reboot' &>/dev/null;
EOF

## 4.5. Print the SSH Connection String for the VM

In [ ]:
%%bash --out sshstring

# If needed, use the output command to SSH into the VM
echo ssh -i ~/.ssh/$iotDeviceId"_id_rsa" $userName@$vm_dns_name

In [ ]:
# save the ssh conn string into .env file
from dotenv import set_key, get_key, find_dotenv
envPath = find_dotenv(raise_error_if_not_found=True)

set_key(envPath, "sshstring", sshstringsshstring)